# SID Genetics Study Step 2: Assigning Covariates for SID Phenotype

## Objective
The purpose of this notebook is to assign covariates to eligible statin users and non-users as part of the statin-induced diabetes (SID) phenotype. Covariates assigned include demographic characteristics (self-identified race, ethnicity, and sex at birth and age at index) and other baseline diabetes risk factors:
- low high-density lipoprotein (HDL): HDL ≤40 for males and HDL ≤ 50 for females
- high triglycerides (TG): TG ≥ 150
- high body mass index (BMI): BMI ≥ 25
- smoking status: has smoking observation in EHR
- hypertension (HTN) status: has at least 2 HTN ICD codes to prevent misdiagnosis
- prediabetes status: fasting glucose from 100-125mg/dL, random glucose from 140-199mg/dL, Hba1c between 5.7-6.4%
- gestational diabetes status: has at least one ICD code

# Pulling in the Data

**Objective**: The purpose of this section is to load packages and pull in data from the All of Us Research Project (AoURP). AoURP dataset code (R and SQL) is generated using the AoURP's cohort builder.

In [ ]:
# Demographics (Sex & Race/Ethnicity) SQL

library(tidyverse)
library(bigrquery)

# This query represents dataset "All Subjects Covariate" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_75324549_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_75324549_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_75324549",
  "person_75324549_*.csv")
message(str_glue('The data will be written to {person_75324549_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_75324549_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_75324549_path,
  destination_format = "CSV")

# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_75324549_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_75324549_person_df <- read_bq_export_from_workspace_bucket(person_75324549_path)

dim(dataset_75324549_person_df)

# head(dataset_75324549_person_df, 5)

In [ ]:
# Gestational Diabetes, Prediabetes, HTN SQL - dataset_75324549_condition_df

library(tidyverse)
library(bigrquery)

# This query represents dataset "All Subjects Covariate" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_75324549_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (1571690, 1571691, 35207668, 37200977, 37200978, 37200979, 37201113, 44821949, 44822099, 44822104, 44823109, 44823246, 44823247, 44829117, 44830221, 44831389, 44831390, 44832532, 44832533, 44833556, 44834715, 44836084, 44837245, 45539106, 45553483, 45558215, 45563059, 45572770, 45582459, 45582460, 45582461, 45592198)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_75324549_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_75324549",
  "condition_75324549_*.csv")
message(str_glue('The data will be written to {condition_75324549_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_75324549_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_75324549_path,
  destination_format = "CSV")

# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_75324549_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), condition_type_concept_name = col_character(), stop_reason = col_character(), visit_occurrence_concept_name = col_character(), condition_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), condition_status_source_value = col_character(), condition_status_concept_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_75324549_condition_df <- read_bq_export_from_workspace_bucket(condition_75324549_path)

dim(dataset_75324549_condition_df)

# head(dataset_75324549_condition_df, 5)

In [ ]:
# Filter HTN data frame to make sure patients are only diagnosed with HTN if they have at least 2 ICD codes
htn_icd_df <- dataset_75324549_condition_df[which(grepl(pattern = "I10*|401*", 
                                                        dataset_75324549_condition_df$source_concept_code)),
                                           c('person_id', 'standard_concept_name', 'condition_start_datetime', 
                                             'source_concept_code')]

htn_diagnosis <- data.frame(table(htn_icd_df$person_id))
htn_ids <- htn_diagnosis[htn_diagnosis$Freq > 1,]

htn_icd_filtered_df <- htn_icd_df %>% filter(person_id %in% htn_ids$Var1) %>% 
group_by(person_id) %>% 
arrange(as.Date(condition_start_datetime)) %>% slice_head()

dim(htn_icd_filtered_df)
# head(htn_icd_filtered_df)

In [ ]:
# Get first GD only
gd_code_df <- dataset_75324549_condition_df %>% filter(grepl(pattern = '648\\.8.|648\\.0.|O24\\.4.', 
                                                             condition_source_value )) %>% 
                group_by(person_id) %>% 
                arrange(condition_start_datetime) %>% 
                slice_head()

In [ ]:
# BMI measurements SQL - dataset_33462820_measurement_df

library(tidyverse)
library(bigrquery)

# This query represents dataset "BMI Dataset" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_33462820_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (3038553)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_33462820_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_33462820",
  "measurement_33462820_*.csv")
message(str_glue('The data will be written to {measurement_33462820_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_33462820_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_33462820_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_33462820_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_33462820_measurement_df <- read_bq_export_from_workspace_bucket(measurement_33462820_path)

dim(dataset_33462820_measurement_df)

# head(dataset_33462820_measurement_df)

In [ ]:
# HDL Measurements SQL - dataset_21028286_measurement_df
library(tidyverse)
library(bigrquery)

# This query represents dataset "HDL Measures Dataset" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_21028286_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (40782589)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_21028286_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_21028286",
  "measurement_21028286_*.csv")
message(str_glue('The data will be written to {measurement_21028286_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_21028286_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_21028286_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_21028286_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_21028286_measurement_df <- read_bq_export_from_workspace_bucket(measurement_21028286_path)

dim(dataset_21028286_measurement_df)

# head(dataset_21028286_measurement_df, 5)

In [ ]:
# LDL cholesterol measures - dataset_69758583_measurement_df

library(tidyverse)
library(bigrquery)

# This query represents dataset "LDL Measures" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_69758583_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (40795800)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_69758583_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_69758583",
  "measurement_69758583_*.csv")
message(str_glue('The data will be written to {measurement_69758583_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_69758583_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_69758583_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_69758583_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_69758583_measurement_df <- read_bq_export_from_workspace_bucket(measurement_69758583_path)

dim(dataset_69758583_measurement_df)

# head(dataset_69758583_measurement_df, 5)

In [ ]:
# Glucose and HbA1c Measurements SQL - dataset_79242111_measurement_df
library(tidyverse)
library(bigrquery)

# This query represents dataset "Glucose and HbA1c Measures" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_79242111_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (3000483, 3003309, 3004410, 3004501, 3005673, 3007263, 3037110)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_79242111_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_79242111",
  "measurement_79242111_*.csv")
message(str_glue('The data will be written to {measurement_79242111_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_79242111_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_79242111_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_79242111_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_79242111_measurement_df <- read_bq_export_from_workspace_bucket(measurement_79242111_path)

dim(dataset_79242111_measurement_df)

# head(dataset_79242111_measurement_df, 5)

In [ ]:
# Triglyceride Measures SQL - dataset_60405046_measurement_df
library(tidyverse)
library(bigrquery)

# This query represents dataset "Triglyceride Measures Dataset" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_60405046_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (3022038, 3022192)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_60405046_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_60405046",
  "measurement_60405046_*.csv")
message(str_glue('The data will be written to {measurement_60405046_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_60405046_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_60405046_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_60405046_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_60405046_measurement_df <- read_bq_export_from_workspace_bucket(measurement_60405046_path)

dim(dataset_60405046_measurement_df)

# head(dataset_60405046_measurement_df, 5)

In [ ]:
# Get T2D measures and prediabetes status
library(tidyverse)

# Collect diabetes measures
t2d_meas_tbl <- dataset_79242111_measurement_df %>% 
            select(person_id, measurement_concept_id, value_as_number, measurement_datetime)

# Prediabetes status from labs
library(sqldf)

# Use SQL to collect measurements that meet prediabetes criteria
prediabetes_meas_df <- sqldf("SELECT * FROM t2d_meas_tbl WHERE (measurement_concept_id IN ('3000483', '3004501') AND value_as_number <= 199 AND value_as_number >= 140) OR (measurement_concept_id = '3037110' AND value_as_number < 125 AND value_as_number >= 100) OR (measurement_concept_id IN ('3004410', '3007263', '3003309', '3005673') AND value_as_number >= 5.7 AND value_as_number <= 6.4)")
prediabetes_meas_df <- prediabetes_meas_df %>% 
    group_by(person_id) %>% 
    arrange(as.Date(measurement_datetime)) %>% 
    slice_head() 

dim(prediabetes_meas_df)
# head(prediabetes_meas_df)

In [ ]:
# Data frame of smoking start ages
library(allofus)
con <- aou_connect()

smoking_obs_tbl <- tbl(con, "observation") |> filter(observation_concept_id == 40766333) |>
            select(person_id, value_as_number) |> collect()

# smoking_obs_tbl |> head()
smoking_obs_tbl |> tally()

# Convert person_id variable drawn in using the allofus package to numeric
smoking_obs_tbl$person_id <- as.numeric(smoking_obs_tbl$person_id)

In [ ]:
# Pull in data frames from preparation step
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "statin_rx_df_v2_distinct.csv", " ."), intern=T)
statin_users_eof_df <- read.csv("statin_rx_df_v2_distinct.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "non_users_eof_df_v2.csv", " ."), intern=T)
non_users_eof_df <- read.csv("non_users_eof_df_v2.csv")

# Covariates for Statin Users

**Objective**: The purpose of this section is to assign covariate status at baseline (so most recent before statin initiation) for eligible statin users.

In [ ]:
# Get necessary demographic info for statin users
statin_user_dem_df <- inner_join(statin_users_eof_df, dataset_75324549_person_df, by = join_by(person_id)) %>% 
                        select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                               statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                               statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity)

# Check that data frames joined correctly
dim(statin_user_dem_df)
# head(statin_user_dem_df)

# Summarize demographic characteristics for current cohort
# table(statin_user_dem_df$sex_at_birth)
# table(statin_user_dem_df$race)
# table(statin_user_dem_df$ethnicity)

In [ ]:
# Create column for low HDL
statin_user_hdl_df <- left_join(statin_user_dem_df, dataset_21028286_measurement_df) %>% 
                            mutate(low_hdl = case_when((sex_at_birth == "Female" & value_as_number <= 50) | 
                                                   (sex_at_birth == "Male" & value_as_number <= 40) ~ 1,
                                                   (sex_at_birth == "Female" & value_as_number > 50) | 
                                                   (sex_at_birth == "Male" & value_as_number > 40)~ 0
                                                   )) %>% 
                            group_by(person_id) %>% 
                            mutate(low_hdl = ifelse(any(low_hdl == 1), 1, 0)) %>% 
                            ungroup() %>%
                            select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                   statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                   statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl) %>% 
                            distinct(.keep_all = TRUE)

In [ ]:
# Create column for high TG
statin_user_tg_df <- left_join(statin_user_hdl_df, dataset_60405046_measurement_df) %>% 
                            mutate(high_tg = ifelse(value_as_number >= 150 & as.Date(measurement_datetime) < statin_init_date, 1, 0)) %>% 
                            group_by(person_id) %>% 
                            mutate(tg_over_150 = ifelse(any(high_tg == 1), 1, 0)) %>% 
                            ungroup() %>%
                            select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                   statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                   statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl,
                                   tg_over_150) %>% 
                            distinct(.keep_all = TRUE)

In [ ]:
# Create column for high BMI
statin_user_bmi_df <- left_join(statin_user_tg_df, dataset_33462820_measurement_df) %>% 
                            mutate(high_bmi = ifelse(value_as_number >= 25 & as.Date(measurement_datetime) < statin_init_date, 1, 0)) %>% 
                            group_by(person_id) %>% 
                            mutate(bmi_over_25 = ifelse(any(high_bmi == 1), 1, 0)) %>% 
                            ungroup() %>%
                            select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                   statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                   statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl,
                                   tg_over_150, bmi_over_25) %>% 
                            distinct(.keep_all = TRUE)

In [ ]:
# Create column for smoking status
statin_user_smoking_df <- left_join(statin_user_bmi_df, smoking_obs_tbl) %>% 
                                mutate(smokingo_start_age = ifelse(person_id %in% smoking_obs_tbl$person_id, 
                                                               as.numeric(smoking_obs_tbl$value_as_number), NA),
                                       smoking_status = ifelse(!is.na(smokingo_start_age) & smokingo_start_age <= statin_init_age, 1, 0)) %>%
                                select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                       statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                       statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl,
                                       tg_over_150, bmi_over_25, smoking_status)


In [ ]:
# Create column for HTN status
statin_user_htn_icd_df <- left_join(statin_user_smoking_df, htn_icd_filtered_df) %>% 
                                mutate(htn_status = ifelse(!is.na(source_concept_code) & as.Date(condition_start_datetime) < statin_init_date, 1, 0)) %>%
                                select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                       statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                       statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl, 
                                       tg_over_150, bmi_over_25, smoking_status, htn_status)

In [ ]:
# Create column for prediabetes based on lab values
statin_user_pd_df <- left_join(statin_user_htn_icd_df, prediabetes_meas_df) %>% 
                            filter(person_id %in% statin_user_dem_df$person_id) %>% 
                            mutate(pd_status = ifelse(!is.na(measurement_concept_id) & 
                                   as.Date(measurement_datetime) < statin_init_date, 1, 0)) %>%
                            select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                   statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                   statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl, 
                                   tg_over_150, bmi_over_25, smoking_status, htn_status, pd_status) %>%
                            distinct(.keep_all = TRUE)

In [ ]:
# Create column for gestational diabetes based on ICD codes
statin_user_gd_df <- left_join(statin_user_pd_df, gd_code_df) %>%
                            mutate(gd_status = ifelse(!is.na(standard_concept_code) & as.Date(condition_start_datetime) < statin_init_date, 1, 0)) %>%
                            select(person_id, t2d_status, eof_age, eof_datetime, statin_init_age, statin_init_date, 
                                   statin_type_start, statin_dose_start, statin_end_date, statin_end_age, 
                                   statin_type_end, statin_dose_end, sex_at_birth, race, ethnicity, low_hdl, 
                                   tg_over_150, bmi_over_25, smoking_status, htn_status, pd_status, gd_status) %>%
                            distinct(.keep_all = TRUE)

In [ ]:
# Copy GD table to get final statin user with covariates table
statin_user_covs_df <- statin_user_gd_df

# Check final table
length(unique(statin_user_covs_df$person_id))
dim(statin_user_covs_df)
# head(statin_user_covs_df)

In [ ]:
# Summarize how many missing status' exist for each covariate
# statin_user_covs_df %>% summarize(across(everything(), ~ sum(is.na(.x))))

In [ ]:
# Save statin user covariates into workspace bucket
write.csv(statin_user_covs_df, "statin_user_covs_df_v2.csv")
system(paste0("gsutil cp ./", "statin_user_covs_df_v2.csv", " ", my_bucket, "/sid_pheno_files/"), intern=T)

# Covariates for Non-users

**Objective**: The purpose of this section is to assign statin non-users covariate statuses at every possible index date. Since the next step involves matching 2 non-users to each statin user, we want to create a larger pool of non-users to choose from by having multiple possible start dates for each non-user. Dates where non-users received fasting glucose, random glucose, or HbA1c measurements are considered eligible index dates, and thus, for non-users to be eligible for this study, they must have at least one of these measures.

**Note**: Due to the large amount of data being handled in this section, the kernel is prone to dying. Data frames are saved at multiple checkpoints so that if kernel death does occur, analysis can be restarted at any of the checkpoints.

In [ ]:
# Get T2D measures for eligible non-users since our index dates are based on T2D measures
t2d_meas_tbl <- t2d_meas_tbl %>% filter(person_id %in% non_users_eof_df$person_id)

# Get all random glucose/fasting glucose/HbA1c measures for non-users 
non_user_meas_df <- right_join(t2d_meas_tbl, non_users_eof_df, by = 'person_id')

# Convert dates to characters for compatibility
non_user_meas_df$eof_date <- as.character(non_user_meas_df$eof_datetime)
non_user_meas_df$measurement_datetime <- as.character(non_user_meas_df$measurement_datetime)

# Check starting non-user data frame
length(unique(non_user_meas_df$person_id))
dim(non_user_meas_df)
# head(non_user_meas_df)

In [ ]:
# Filter out index dates that occur after EoF or less than 30 days before EoF
non_user_meas_filtered_df <- non_user_meas_df %>% filter(as.numeric(difftime(as.Date(eof_datetime), 
                                                                             as.Date(measurement_datetime),
                                                                    units = 'days')) >= 30)

# Check filtered non-user data frame
length(unique(non_user_meas_filtered_df$person_id))
dim(non_user_meas_filtered_df)
# head(non_user_meas_filtered_df)

In [ ]:
# Set seed to pick self-identified race, ethnicity, and sex at birth for non-users randomly since they 
# shouldn't change
set.seed(32)
# Get necessary demographic info for non users
non_user_dem_df <- inner_join(non_user_meas_filtered_df, dataset_75324549_person_df, 
                              by = c("person_id", "date_of_birth")) %>%
                        mutate(index_id = sample(1:nrow(cur_data()), nrow(cur_data()))) %>%
                        select(person_id, date_of_birth, t2d_status, eof_age, eof_datetime, sex_at_birth, 
                               race, ethnicity, measurement_concept_id, value_as_number, measurement_datetime, 
                               index_id)

In [ ]:
# Clean up column names and data types
colnames(non_user_dem_df) <- c('person_id', 'date_of_birth', 't2d_status', 'eof_age', 'eof_datetime', 
                                    'sex_at_birth', 'race', 'ethnicity', 't2d_measurement_concept_id', 
                                    't2d_value_as_number', 't2d_measurement_index_date', 'index_id')

non_user_dem_df$eof_date <- as.Date(non_user_dem_df$eof_datetime)
non_user_dem_df$t2d_measurement_index_date <- as.Date(non_user_dem_df$t2d_measurement_index_date)
dim(non_user_dem_df)
# head(non_user_dem_df)

# Save DF in case of kernel death
write.csv(non_user_dem_df, "non_user_dem_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_dem_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Create a skeleton data frame that only contains IDs and index dates
helper_df <- non_user_dem_df %>%  
                select(c('person_id', 't2d_measurement_index_date', 'index_id')) %>% distinct(.keep_all = TRUE)

# Check helper df
length(unique(helper_df$person_id))
dim(helper_df)
# head(helper_df)

In [ ]:
# Determining HDL status with joins - idea provided by lab members

# Create a data frame with HDL measurements for all potential controls
hdl_df <- dataset_21028286_measurement_df[dataset_21028286_measurement_df$person_id %in% 
                                               non_user_dem_df$person_id, c('person_id', 'value_as_number', 
                                                                                 'measurement_datetime')]

# Determine low HDL status ahead of time
hdl_df <- left_join(hdl_df, dataset_75324549_person_df) %>% 
                        select(person_id, value_as_number, measurement_datetime, sex_at_birth) %>%
                        mutate(low_hdl = case_when((sex_at_birth == "Female" & value_as_number <= 50) | 
                                                   (sex_at_birth == "Male" & value_as_number <= 40) ~ 1,
                                                   (sex_at_birth == "Female" & value_as_number > 50) | 
                                                   (sex_at_birth == "Male" & value_as_number > 40)~ 0
                                                   )) %>%
                        arrange(person_id, measurement_datetime) %>% group_by(person_id) %>% 
                        mutate(stretch_group = cumsum(low_hdl != lag(low_hdl, default = first(low_hdl)))) %>%
                        group_by(person_id, low_hdl, stretch_group) %>%
                        summarize(
                            start_date = first(measurement_datetime),
                            end_date = last(measurement_datetime),
                            .groups = "drop"
                                ) %>%
                        select(person_id, low_hdl, start_date, end_date) %>% 
                        arrange(person_id, start_date)

In [ ]:
# Join HDL and demographics data frames and group by person id and index date so that I can find the
# most recent HDL measurement for each index date
non_user_hdl_helper_df <- inner_join(helper_df, hdl_df, relationship = "many-to-many", by = "person_id") %>% 
                            group_by(person_id, t2d_measurement_index_date) %>% 
                            arrange(person_id, as.Date(t2d_measurement_index_date)) %>%
                            filter(as.Date(start_date) <= as.Date(t2d_measurement_index_date)) %>%
                            slice_tail()

# Save DF in case of kernel death
write.csv(non_user_hdl_helper_df, "non_user_hdl_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_hdl_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Determining TG status with joins

# Create a data frame with TG measurements for all potential controls
tg_df <- dataset_60405046_measurement_df[dataset_60405046_measurement_df$person_id %in% 
                                         non_user_dem_df$person_id, c('person_id', 'value_as_number', 
                                                                      'measurement_datetime')]

# Determine low TG status ahead of time
tg_df <- tg_df %>% 
            mutate(high_tg = ifelse(value_as_number >= 150, 1, 0)) %>% 
            arrange(person_id, as.Date(measurement_datetime)) %>% group_by(person_id) %>% 
            mutate(stretch_group = cumsum(high_tg != lag(high_tg, default = first(high_tg)))) %>%
            group_by(person_id, high_tg, stretch_group) %>%
            summarize(
                start_date = first(as.Date(measurement_datetime)),
                end_date = last(measurement_datetime),
                .groups = "drop"
                    ) %>%
            select(person_id, high_tg, start_date, end_date) %>% 
            arrange(person_id, start_date)

In [ ]:
# Join TG and demographics data frames and group by person id and index date so that I can find the
# most recent TG measurement for each index date
non_user_tg_helper_df <- inner_join(helper_df, tg_df, relationship = "many-to-many", by = "person_id") %>% 
                            group_by(person_id, t2d_measurement_index_date) %>% 
                            arrange(person_id, t2d_measurement_index_date) %>%
                            filter(start_date <= t2d_measurement_index_date) %>%
                            slice_tail()

# Save DF in case of kernel death
write.csv(non_user_tg_helper_df, "non_user_tg_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_tg_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Determining high BMI status with joins

# Create a data frame with BMI measurements for all potential controls
bmi_df <- dataset_33462820_measurement_df[dataset_33462820_measurement_df$person_id %in% 
                                          non_user_dem_df$person_id, c('person_id', 'value_as_number', 
                                                                       'measurement_datetime')]

# Determine high BMI status ahead of time
bmi_df <- bmi_df %>% 
                mutate(high_bmi = ifelse(value_as_number >= 25, 1, 0)) %>% 
                arrange(person_id, measurement_datetime) %>% group_by(person_id) %>% 
                mutate(stretch_group = cumsum(high_bmi != lag(high_bmi, default = first(high_bmi)))) %>%
                group_by(person_id, high_bmi, stretch_group) %>%
                summarize(
                    start_date = first(measurement_datetime),
                    end_date = last(measurement_datetime),
                    .groups = "drop"
                        ) %>%
                select(person_id, high_bmi, start_date, end_date) %>% 
                arrange(person_id, start_date)

In [ ]:
# Join BMI and demographics data frames and group by person id and index date so that I can find the
# most recent BMI measurement for each index date
non_user_bmi_helper_df <- inner_join(helper_df, bmi_df, relationship = "many-to-many", by = "person_id") %>% 
                            group_by(person_id, t2d_measurement_index_date) %>% 
                            arrange(person_id, t2d_measurement_index_date) %>%
                            filter(start_date <= t2d_measurement_index_date) %>%
                            slice_tail()

# Save DF in case of kernel death
write.csv(non_user_bmi_helper_df, "non_user_bmi_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_bmi_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Create prediabetes helper table using SQL
library(sqldf)
non_user_dem_df <- non_user_dem_df %>% 
                    mutate(t2d_measurement_concept_id = as.numeric(t2d_measurement_concept_id))

non_user_pd_helper_df <- sqldf("
    SELECT 
        person_id, 
        t2d_measurement_concept_id, 
        t2d_value_as_number, 
        t2d_measurement_index_date, 
        index_id,
        CASE 
            WHEN (t2d_measurement_concept_id IN ('3000483', '3004501') AND t2d_value_as_number <= 199 AND t2d_value_as_number >= 140) 
              OR (t2d_measurement_concept_id = '3037110' AND t2d_value_as_number < 125 AND t2d_value_as_number >= 100) 
              OR (t2d_measurement_concept_id IN ('3004410', '3007263', '3003309', '3005673') AND t2d_value_as_number >= 5.7 AND t2d_value_as_number <= 6.4) 
            THEN 1 
            ELSE 0 
        END AS pd_status 
    FROM 
        non_user_dem_df
    ") %>% 
    select(person_id, t2d_measurement_index_date, index_id, pd_status) %>%
    group_by(person_id, t2d_measurement_index_date) %>% 
    arrange(person_id, t2d_measurement_index_date)

# Save DF in case of kernel death
write.csv(non_user_pd_helper_df, "non_user_pd_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_pd_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Determining smoking status

# Create a data frame with smoking observations for all potential controls
smoking_df <- smoking_obs_tbl[smoking_obs_tbl$person_id %in% non_user_dem_df$person_id, ] %>% 
                        rename(smoking_start_age = value_as_number) %>%
                        mutate_at(vars(person_id), as.numeric)


library(lubridate)
# Find date of smoking start
smoking_df <- full_join(non_user_dem_df, smoking_df) %>% 
                    mutate(smoking_start_date = as.Date(date_of_birth) %m+% years(smoking_start_age)) %>%
                    select(person_id, smoking_start_date) %>% distinct(.keep_all = TRUE)

# Find smoking status at each index date
non_user_smoking_helper_df <- inner_join(helper_df, smoking_df) %>% 
                                    mutate(smoking_status = ifelse(!is.na(smoking_start_date) & 
                                                                   smoking_start_date <= t2d_measurement_index_date, 
                                                                   1, 0)) %>%
                                    select(person_id, t2d_measurement_index_date, index_id, smoking_status)

# Save DF in case of kernel death
write.csv(non_user_smoking_helper_df, "non_user_smoking_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_smoking_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Determining hypertension (HTN) status

# Create a data frame with HTN ICD codes for all potential controls
htn_df <- htn_icd_filtered_df[htn_icd_filtered_df$person_id %in% non_user_dem_df$person_id, ]

# Find HTN status at each index date
non_user_htn_helper_df <- full_join(helper_df, htn_df) %>% 
                            mutate(htn_status = ifelse(!is.na(condition_start_datetime) & 
                                                       as.Date(condition_start_datetime) <= t2d_measurement_index_date, 
                                                       1, 0)) %>%
                            select(person_id, t2d_measurement_index_date, index_id, htn_status)

# Save DF in case of kernel death
write.csv(non_user_htn_helper_df, "non_user_htn_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_htn_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
# Determining gestational diabetes (GD) status

# Create a data frame with GD ICD codes for all potential controls
gd_df <- gd_code_df[gd_code_df$person_id %in% non_user_dem_df$person_id, ]

# Find GD status at each index date
non_user_gd_helper_df <- full_join(helper_df, gd_df) %>% 
                            mutate(gd_status = ifelse(!is.na(condition_start_datetime) & 
                                                      as.Date(condition_start_datetime) <= t2d_measurement_index_date, 
                                                      1, 0)) %>%
                            select(person_id, t2d_measurement_index_date, index_id, condition_start_datetime, 
                                   gd_status)

# Save DF in case of kernel death
write.csv(non_user_gd_helper_df, "non_user_gd_helper_df_v2.csv")
system(paste0("gsutil cp ./", "non_user_gd_helper_df_v2.csv", " ", my_bucket, "/sid_pheno_files/covariate_helpers/"), intern=T)

In [ ]:
library(allofus)
library(tidyverse)
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Reload data frames
system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_dem_df_v2.csv", " ."), intern=T)
non_user_dem_df <- read.csv("non_user_dem_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_hdl_helper_df_v2.csv", " ."), intern=T)
non_user_hdl_helper_df <- read.csv("non_user_hdl_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_tg_helper_df_v2.csv", " ."), intern=T)
non_user_tg_helper_df <- read.csv("non_user_tg_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_bmi_helper_df_v2.csv", " ."), intern=T)
non_user_bmi_helper_df <- read.csv("non_user_bmi_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_pd_helper_df_v2.csv", " ."), intern=T)
non_user_pd_helper_df <- read.csv("non_user_pd_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_smoking_helper_df_v2.csv", " ."), intern=T)
non_user_smoking_helper_df <- read.csv("non_user_smoking_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_htn_helper_df_v2.csv", " ."), intern=T)
non_user_htn_helper_df <- read.csv("non_user_htn_helper_df_v2.csv")

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/covariate_helpers/", "non_user_gd_helper_df_v2.csv", " ."), intern=T)
non_user_gd_helper_df <- read.csv("non_user_gd_helper_df_v2.csv")

In [ ]:
# Check status counts for each covariate
table(non_user_hdl_helper_df$low_hdl, useNA = 'always')
table(non_user_bmi_helper_df$high_bmi, useNA = 'always')
table(non_user_tg_helper_df$high_tg, useNA = 'always')
table(non_user_pd_helper_df$pd_status, useNA = 'always')
table(non_user_smoking_helper_df$smoking_status, useNA = 'always')
table(non_user_htn_helper_df$htn_status, useNA = 'always')
table(non_user_gd_helper_df$gd_status, useNA = 'always')

In [ ]:
# Check data frames
# head(non_user_dem_df)
# head(non_user_hdl_helper_df)

In [ ]:
# Combine helper data frames into one main data frame
library(tidyverse)

# HDL
non_user_covs_df <- left_join(non_user_dem_df, non_user_hdl_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl)

In [ ]:
# TG
non_user_covs_df <- left_join(non_user_covs_df, non_user_tg_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg)

In [ ]:
# BMI
non_user_covs_df <- left_join(non_user_covs_df, non_user_bmi_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg, high_bmi)

In [ ]:
# PD status
non_user_covs_df <- left_join(non_user_covs_df, non_user_pd_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg, high_bmi, pd_status) %>% 
                        distinct(.keep_all = TRUE)

In [ ]:
# Smoking status
non_user_covs_df <- left_join(non_user_covs_df, non_user_smoking_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg, high_bmi, pd_status, 
                               smoking_status)

In [ ]:
# HTN status
non_user_covs_df <- left_join(non_user_covs_df, non_user_htn_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg, high_bmi, pd_status, 
                               smoking_status, htn_status)

In [ ]:
# GD status
non_user_covs_df <- left_join(non_user_covs_df, non_user_gd_helper_df, 
                              by = join_by(person_id, t2d_measurement_index_date, index_id)) %>% 
                        arrange(person_id, t2d_measurement_index_date) %>%
                        select(person_id, t2d_status, date_of_birth, eof_age, eof_date, sex_at_birth, race, 
                               ethnicity, t2d_measurement_concept_id, t2d_value_as_number, 
                               t2d_measurement_index_date, index_id, low_hdl, high_tg, high_bmi, pd_status, 
                               smoking_status, htn_status, gd_status)

In [ ]:
# Check how much data is missing
non_user_covs_df %>% summarize(na_low_hdl = sum(is.na(low_hdl)),
                                         na_high_tg = sum(is.na(high_tg)),
                                         na_high_bmi = sum(is.na(high_bmi)),
                                         na_pd_status = sum(is.na(pd_status)),
                                         na_smoking_status = sum(is.na(smoking_status)),
                                         na_htn_status = sum(is.na(htn_status)),
                                         na_gd_status = sum(is.na(gd_status)))

In [ ]:
# Save non-user covariates df into workspace bucket
write.csv(non_user_covs_df, "non_users_covs_df_v2.csv")
system(paste0("gsutil cp ./", "non_users_covs_df_v2.csv", " ", my_bucket, "/sid_pheno_files/"), intern=T)